In [2]:
!pip install ultralytics

  Obtaining dependency information for ultralytics from https://files.pythonhosted.org/packages/58/df/b44a846a103476fe9b4f9bb0fe99cb787fecb48ee795c2b2c4578ca98217/ultralytics-8.1.27-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.3 kB ? eta -:--:--
     ---------- ----------------------------- 10.2/40.3 kB ? eta -:--:--
     ------------------- ------------------ 20.5/40.3 kB 165.2 kB/s eta 0:00:01
     ---------------------------- --------- 30.7/40.3 kB 187.9 kB/s eta 0:00:01
     -------------------------------------- 40.3/40.3 kB 160.9 kB/s eta 0:00:00
  Obtaining dependency information for torchvision>=0.9.0 from https://files.pythonhosted.org/packages/e7/45/419aa0b37254f1fd62b45bb63836066c5eb81e37d70940e0491e95167eed/torchvision-0.17.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for thop>=0.1.1 from https://files.pythonhosted.org/packages/bb/0f/72beeab4

In [4]:
!pip install cvzone

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26310 sha256=f7da7acb5a45d7210595ca73bb1c94b4f57b064c0b630858de648890e4cbd8eb
  Stored in directory: c:\users\matangee\appdata\local\pip\cache\wheels\ab\36\ec\47be2d4e59dc4289e684d5b0dde54d1e72e51a614e57690e85
Successfully built cvzone


In [5]:
from ultralytics import YOLO
import cv2
import cvzone
import math

In [ ]:
cap = cv2.VideoCapture(r"C:\Users\MATANGEE\Downloads\Personal-Protective-Equipment-PPE-Detection-main (1)\Personal-Protective-Equipment-PPE-Detection-main\ppe.mp4")

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
writer = cv2.VideoWriter('Output.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 25, (width, height))

model = YOLO("ppe.pt")

classNames = ['Hardhat', 'Mask', 'NO-Hardhat', 'NO-Mask', 'NO-Safety Vest', 'Person', 'Safety Cone',
              'Safety Vest', 'machinery', 'vehicle']
myColor = (0, 0, 255)
while True:
    ret, img = cap.read()
    if not ret:
        print("Can't retrieve img - stream may have ended. Exiting..")
        break
    results = model(img, stream=True)
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
            print(currentClass)
            if conf > 0.5:
                if currentClass == 'NO-Hardhat' or currentClass == 'NO-Safety Vest' or currentClass == "NO-Mask":
                    myColor = (0, 0, 255)
                elif currentClass == 'Hardhat' or currentClass == 'Safety Vest' or currentClass == "Mask":
                    myColor = (0, 255, 0)
                else:
                    myColor = (255, 0, 0)

                cvzone.putTextRect(img, f'{classNames[cls]} {conf}',
                                   (max(0, x1), max(35, y1)), scale=1, thickness=1, colorB=myColor,
                                   colorT=(255, 255, 255), colorR=myColor, offset=5)
                cv2.rectangle(img, (x1, y1), (x2, y2), myColor, 3)

    writer.write(img)
    cv2.imshow("PPE Detection", img)
    if cv2.waitKey(10) & 0xFF == ord("q"):
        break
    
cap.release()
writer.release()
cv2.destroyAllWindows()